In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [1]:
areas = ('VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam') # original name in allen brain observatory
areas_map = {'VISp':'V1', 'VISl':'LM', 'VISrl':'RL', 'VISal':'AL', 'VISpm':'PM', 'VISam':'AM'} # shortened name commonly used in other papers

## Load in sessions where the activity of the 6 visual cortex areas of interest were measured

In [4]:
# Example cache directory path, it determines where downloaded data will be stored
output_dir = '/home/dan/Desktop/AllenSDKCache'

In [5]:
# this path determines where downloaded data will be stored
manifest_path = os.path.join(output_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
print(cache.get_all_session_types())

['brain_observatory_1.1', 'functional_connectivity']


In [6]:
sessions = cache.get_session_table()
brain_observatory_type_sessions = sessions[sessions["session_type"] == "brain_observatory_1.1"]
brain_observatory_type_sessions

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."
744228101,2019-10-03T00:00:00Z,719817805,brain_observatory_1.1,122.0,M,wt/wt,659,2226,6,"[Eth, TH, LP, POL, APN, DG, CA1, VIS, nan, CA3..."


In [7]:
valid_session_ids = []
ephys_structure_acronyms = brain_observatory_type_sessions['ecephys_structure_acronyms']
for id, acronyms in ephys_structure_acronyms.items():
    if set(areas).issubset(acronyms):
        valid_session_ids.append(id)

print("Found the following valid sessions:")
print(valid_session_ids)

Found the following valid sessions:
[719161530, 750332458, 750749662, 754312389, 755434585, 756029989, 791319847, 797828357]


## Extract the relevant data from each session
This will be 100 seconds of activity from 10 seconds before, and 90 seconds after a stimulus is presented to the mouse. There are multiple distinct stimulus presentation times (typicially 15-20) in each session. For each presentation, we extract the activity of the 6 visual cortex areas of interest. The raw data contains spike timings, we bin these into 100ms bins for each neuron. The output for a given area and a given presentation is a 1000*d matrix, representing the activity of d neurons in 100ms bins for 100 seconds (1000 timsteps).

In [8]:
session_id = valid_session_ids[5] # choose one of the valid sessions
session = cache.get_session_data(session_id)

/home/dan/anaconda3/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/home/dan/anaconda3/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  return func(args[0], **pargs)


In [9]:
# find when the stimulus times occur
stimulus_epochs = session.get_stimulus_epochs()
stimulus_epochs

,start_time,stop_time,duration,stimulus_name,stimulus_block
0,24.429348,84.496188,60.066840,spontaneous,null
1,84.496188,996.491813,911.995625,gabors,0.0
2,996.491813,1285.483398,288.991585,spontaneous,null
3,1285.483398,1583.982946,298.499548,flashes,1.0
4,1583.982946,1585.734418,1.751472,spontaneous,null
5,1585.734418,2185.235561,599.501143,drifting_gratings,2.0
6,2185.235561,2216.261498,31.025937,spontaneous,null
7,2216.261498,2816.763498,600.502000,natural_movie_three,3.0
8,2816.763498,2846.788598,30.025100,spontaneous,null
9,2846.788598,3147.039578,300.250980,natural_movie_one,4.0


In [10]:
# filter out the spontaneous and invalid presentations
stim_start_times = stimulus_epochs[(stimulus_epochs['stimulus_name'] != 'spontaneous') & (stimulus_epochs['stimulus_name'] != 'invalid_presentation')][['start_time']]
stim_start_times = list(stim_start_times['start_time'])
print("Found the following", len(stim_start_times) ,"stimulus start times:")
print(stim_start_times)

Found the following 15 stimulus start times:
[84.49618798636119, 1285.4833979863608, 1585.7344179863614, 2216.2614979863615, 2846.7885979863613, 3177.0646879863616, 4077.8343479863615, 4708.361437986361, 5398.938717986361, 5909.365397986361, 6690.017947986361, 7200.444627986361, 7710.871347986361, 8041.147407986361, 8611.624247986361]


In [11]:
# check that the areas of interest indeed have neurons recorded
for area_idx, area in enumerate(areas):
    units = session.units[(session.units.ecephys_structure_acronym == area)].index.values
    print(areas_map[area], 'has', len(units), 'units')

V1 has 51 units
LM has 30 units
RL has 24 units
AL has 51 units
PM has 90 units
AM has 72 units


In [91]:
dataset = {areas_map[area]:[] for area in areas} # dataset is keyed by shortened area name

for start_time in stim_start_times:
    print('> Collecting data for Stimulus presented at time:', start_time)
    sample_start_time = start_time - 10 # go back 10 seconds
    sample_end_time = start_time + 90 # go forward 90 seconds
    
    for area_idx, area in enumerate(areas):
        print(areas_map[area])
        # get the units in the specified area
        units = session.units[(session.units.ecephys_structure_acronym == area)].index.values
        
        firing_rates = []
        for idx, unit in enumerate(units):
            # get spike times for the specified unit
            spike_data = session.spike_times[unit]
            # only keep spikes in the specified time window
            spike_data = spike_data[sample_start_time <= spike_data]
            spike_data = spike_data[spike_data <= sample_end_time]
            spike_data = spike_data - sample_start_time
            # count the number of spikes ocurring every 100 ms
            bins = np.arange(0, 100.1, 0.1)
            binned_spike_counts, _ = np.histogram(spike_data, bins=bins)
            firing_rates.append(binned_spike_counts)

        firing_rates = np.array(firing_rates)
        firing_rates = firing_rates.T
        print(firing_rates.shape)

        dataset[areas_map[area]].append(firing_rates)

> Collecting data for Stimulus presented at time: 84.49618798636119
V1
(1000, 51)
LM
(1000, 30)
RL
(1000, 24)
AL
(1000, 51)
PM
(1000, 90)
AM
(1000, 72)
> Collecting data for Stimulus presented at time: 1285.4833979863608
V1
(1000, 51)
LM
(1000, 30)
RL
(1000, 24)
AL
(1000, 51)
PM
(1000, 90)
AM
(1000, 72)
> Collecting data for Stimulus presented at time: 1585.7344179863614
V1
(1000, 51)
LM
(1000, 30)
RL
(1000, 24)
AL
(1000, 51)
PM
(1000, 90)
AM
(1000, 72)
> Collecting data for Stimulus presented at time: 2216.2614979863615
V1
(1000, 51)
LM
(1000, 30)
RL
(1000, 24)
AL
(1000, 51)
PM
(1000, 90)
AM
(1000, 72)
> Collecting data for Stimulus presented at time: 2846.7885979863613
V1
(1000, 51)
LM
(1000, 30)
RL
(1000, 24)
AL
(1000, 51)
PM
(1000, 90)
AM
(1000, 72)
> Collecting data for Stimulus presented at time: 3177.0646879863616
V1
(1000, 51)
LM
(1000, 30)
RL
(1000, 24)
AL
(1000, 51)
PM
(1000, 90)
AM
(1000, 72)
> Collecting data for Stimulus presented at time: 4077.8343479863615
V1
(1000, 51)


In [92]:
# Use pickle.dump() to serialize the dictionary and write it to the file
with open(f'session_{session_id}_100ms.pkl', 'wb') as file:
    #pickle.dump(dataset, file)